In [ ]:
from os import chdir
chdir('/home/shoaib/Thesis/dmdt_Analysis/')
from dmdt_functions import get_differenciation, get_2Dhistogram

In [ ]:
from pandas import read_parquet, DataFrame
from numpy import linspace

In [ ]:
filtered_lightcurves = "/home/shoaib/PSChallenge/filtered_lightcurves.parquet"

xbins, ybins = 51, 51
# These files do'nt exist yet, they are save paths for later in the notebook
dmdt_by_oid_50x50 = '/home/shoaib/Thesis/dmdt_Analysis/Histograms/dmdt_by_OID_50x50.pkl'

In [ ]:
### By experimentation we know that these are good bins
log_dt_bins = linspace(-4, 3.5, xbins)
dm_bins = linspace(-0.85, 0.8, ybins)

all_lightcurves = read_parquet(filtered_lightcurves)

In [ ]:
grouped = all_lightcurves.groupby('oid_alerce')
num_total_oids = len(grouped)
count = 0
histogram_dict_list = []

for oid, df in grouped:
    df = df.sort_values(by='mjd')
    
    count += 1
    print(f'{count}\t/ {num_total_oids} OIDs done.')
    
    times, mags = df['mjd'].to_numpy(), df['mag'].to_numpy()
    dtimes, dmags = get_differenciation(times=times, magnitudes=mags)
    hist, _junk, _junk = get_2Dhistogram(dtimes=dtimes, dmagnitudes=dmags, dt_bins=log_dt_bins, dm_bins=dm_bins, normalise=False, scale_factor=False)

    if hist.sum() == 0:
        continue
    
    hist_normalised = hist / hist.sum()
    name = df['name'].iloc[0]
    object_type = df['type'].iloc[0]
    band = df['band'].iloc[0]
    n_good_det = len(df)
    
    histogram_dict = {'oid_alerce': oid, 'name': name, 'type': object_type, 'band': band, 'n_good_det': n_good_det,
                      'histogram': hist, 'histogram_normalised': hist_normalised}
    histogram_dict_list.append(histogram_dict)

In [ ]:
hist_df = DataFrame(histogram_dict_list)
hist_df = hist_df.sort_values(by=['type', 'name', 'band'], ignore_index=True)
hist_df.reset_index(drop=True).to_pickle(dmdt_by_oid_50x50)

In [ ]:
hist_df